In [ ]:
import numpy as np
import cv2
import os
from glob import glob
from tqdm import tqdm
import random
import torch
import torchvision.transforms as TF
import csv
from typing import no_type_check
from sklearn import mixture
from sklearn.cluster import KMeans

### making shadow mask

In [ ]:
NAME = DIR = "Jung"

shadowfree_img_paths = sorted(glob(f"../datasets/{NAME}/shadowfree_image/*"))
shadow_img_paths = sorted(glob(f"../datasets/{NAME}/shadow_image/*"))

if not os.path.exists(f"../datasets/{NAME}/shadow_mask/"):
    os.mkdir(f"../datasets/{NAME}/shadow_mask/")

NUMS = len(shadow_img_paths)

for idx in tqdm(range(NUMS)):

    shadowfree_img_path = shadowfree_img_paths[idx]
    shadow_img_path = shadow_img_paths[idx]

    img_name = shadow_img_path.split("/")[-1]

    shadowfree_img = cv2.imread(shadowfree_img_path).astype(np.float64)
    shadow_img = cv2.imread(shadow_img_path).astype(np.float64)

    diff = (shadowfree_img-shadow_img)
    diff = ((diff-diff.min())/(diff.max()-diff.min())*255).astype(np.uint8)

    gray = cv2.cvtColor(diff, cv2.COLOR_RGB2GRAY)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
    cv2.imwrite(os.path.join(f"../datasets/{NAME}/shadow_mask/", img_name), th)

# making rgb color

In [ ]:
def get_average_color(x):
    b, g, r = x[:, 0], x[:, 1], x[:, 2]

    return np.array([np.mean(b), np.mean(g), np.mean(r)])

In [42]:
shadowfree_path = sorted(glob(f"../datasets/{DIR}/shadowfree_image/*.jpg"))
save_path = f"../datasets/{DIR}/background_color_csv"
save_path_img = f"../datasets/{DIR}/background_color_image"
print("the number of images: ", len(shadowfree_path))

kernel = (5, 5)

if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(save_path_img):
    os.mkdir(save_path_img)

mode = "kmeans"

for path in tqdm(shadowfree_path):
    file_name = path.split("/")[-1]

    x = cv2.imread(path)
    x = cv2.resize(x, (512, 512))
    h, w, c = x.shape
    x = x.flatten().reshape(h*w, c)

    if mode == "gmm":
        model = mixture.GaussianMixture(n_components=2, covariance_type='full')
        model.fit(x)
    elif mode == "kmeans":
        model = KMeans(n_clusters=2)
        model.fit(x)

    cls = model.predict(x.flatten().reshape(h*w, c))
    cls0_colors = x[cls == 0]
    cls1_colors = x[cls == 1]

    cls0_avg_color = get_average_color(cls0_colors)
    cls1_avg_color = get_average_color(cls1_colors)

    if np.sum(cls0_avg_color)>=np.sum(cls1_avg_color):
        out_color = cls0_avg_color
    else:
        out_color = cls1_avg_color


    cv2.imwrite(os.path.join(save_path_img, file_name), np.full_like(x, out_color).reshape(h, w, c))

    with open(os.path.join(save_path, file_name.split(".")[0] + ".csv"), "w") as f:
        writer = csv.writer(f)
        writer.writerow(out_color)

the number of images:  237


100%|██████████| 237/237 [08:54<00:00,  2.26s/it]
